In [1]:
from hsflfm.processing import Aligner, StrikeProcessor
from hsflfm.util import MetadataManager, load_dictionary, save_dictionary

import os
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
save_folder = "temporary_result_storage"
assert os.path.exists(save_folder)

saved_results_filename = save_folder + "/processed.json"
if not os.path.exists(saved_results_filename):
    processed_vids = {}
else:
    processed_vids = load_dictionary(saved_results_filename)

In [4]:
specimen_numbers = MetadataManager.all_specimen_numbers()

In [5]:
for i, num in enumerate(specimen_numbers):
    print(num, f"specimen {i} of {len(specimen_numbers)}")
    try:
        data_manager = MetadataManager(num)
        aligner = Aligner(num)

        strike_numbers = data_manager.strike_numbers

        f = save_folder + f"/{num}"
        if not os.path.exists(f):
            os.mkdir(f)

        if num not in processed_vids:
            processed_vids[num] = []
    except Exception as e:
        print(f"failed on specimen {num}, {e}")

    for strike_num in tqdm(strike_numbers):
        if strike_num in processed_vids[num]:
            # print(f"skipping {num} strike {strike_num}")
            continue
        try:
            result_info = aligner.prepare_strike_results(strike_num)
            processor = StrikeProcessor(result_info)
            processor.get_flow_vectors()
            processor.run_regression()
            processor.get_relative_displacements()
            result_info = processor.condense_info()

            save_name = f + f"/strike_{strike_num}_results.json"
            save_dictionary(result_info, save_name)
            assert os.path.exists(save_name)
            processed_vids[num].append(int(strike_num))
            save_dictionary(processed_vids, saved_results_filename)
        except Exception as e:
            print(f"""failed on specimen {num} strike {strike_num}, {e}""")

20220422_OB_1 specimen 0 of 29


100%|██████████| 7/7 [01:41<00:00, 14.55s/it]


20220427_OB_1 specimen 1 of 29


100%|██████████| 8/8 [02:09<00:00, 16.23s/it]


20220427_OB_2 specimen 2 of 29


...

100%|██████████| 15/15 [04:03<00:00, 16.23s/it]


20240507_OB_2 specimen 27 of 29


100%|██████████| 16/16 [06:28<00:00, 24.30s/it]


20240507_OB_3 specimen 28 of 29


100%|██████████| 10/10 [01:14<00:00,  7.42s/it]
